<a href="https://colab.research.google.com/github/Vveanta/Vveanta/blob/main/Google_News_Sentimental_Analyses_for_News_Related_to_Narendra_Modi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from bs4 import BeautifulSoup
import urllib.request,urllib.parse,urllib.error
import requests
import time
import os

# step 1 lets scrape the guided project website with all the posts:
try:
    os.remove("/content/data.csv")
except:
    pass

root= 'https://www.google.com'
link="https://www.google.com/search?q=Narendra+Modi&tbm=nws&sxsrf=ALiCzsb8_GqnC8dnjUOBQ-0O1eHOVyRYbQ:1659451072593&source=lnt&tbs=qdr:w&sa=X&ved=2ahUKEwiHutD9sKj5AhUU62EKHZN_DLEQpwV6BAgBEBc&biw=759&bih=636&dpr=1"
with open('data.csv',"a") as f:
  f.write('Title,Time,Description,Source\n')
def newsParser(link):
    req=urllib.request.Request(link,headers={'User-Agent':'Chrome/51.0.2704.103'})
    html=urllib.request.urlopen(req).read()
    with requests.Session() as c:
        soup = BeautifulSoup(html, 'html5lib')
    
    # print(soup)
    for item in soup.find_all('div',attrs={'class':'Gx5Zad fP1Qef xpd EtOod pkphOe'}):
        rawlink= (item.find('a',href=True)['href'])
        link=rawlink.split("/url?q=")[1]
        link=link.split('&sa=U&')[0]
        title=item.find('div',attrs={'class':'BNeawe vvjwJb AP7Wnd'}).contents[0]
        title=title.replace(",","")
        des=item.find('div',attrs={'class':'BNeawe s3v9rd AP7Wnd'}).get_text().split(".")
        # print(des)
        desc=des[0]
        desc=desc.replace(",","")
        t=des[-1]
        # print(item)
        # print(title)
        # print(desc)
        # print(time)
        # print(link)
        
        with open('data.csv',"a") as f:
            f.write("{},{},{},{}\n".format(title,t,desc,link))

    next=soup.find('a',attrs={'aria-label':'Next page'})
    next=next['href']
    link=root+next
    newsParser(link)
try:
    newsParser(link)  
except TypeError:
    print("pages finished")  
 
        

pages finished


In [10]:
# print(soup)

In [11]:
pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as Sia

In [13]:
df=pd.read_csv('/content/data.csv')
# print(df.head())
analyzer=Sia()
negative=[]
neutral=[]
positive=[]
aggregate_compound=[]
df=df.dropna()
for i in range(df.shape[0]):    
    # print(i)
    title=df.iloc[i,0]
    description=df.iloc[i,2]
    # print(title)
    # print(description)
    title_analyzed=analyzer.polarity_scores(title)
    desc_analyzed=analyzer.polarity_scores(description)
    # print('title: ',title_analyzed)
    # print('description: ',desc_analyzed)
    negative.append((title_analyzed['neg']+desc_analyzed['neg'])/2)
    positive.append((title_analyzed['pos']+desc_analyzed['pos'])/2)
    neutral.append((title_analyzed['neu']+desc_analyzed['neu'])/2)
    aggregate_compound.append((title_analyzed['compound']+desc_analyzed['compound'])/2)
df["Negative"]=negative
df["Positive"]=positive
df["Neutral"]=neutral
df["Aggregate Compound"]=aggregate_compound
pd.set_option('display.max_columns',None)
pd.options.mode.chained_assignment = None
print(df.head())


                                               Title          Time  \
0               What Narendra Modi Is Taking From Me    3 days ago   
1  PM Modi changes display picture to Tricolour a...  15 hours ago   
2  My mother beats me for asking for a pencil: 6-...     1 day ago   
3  PM Modi's brother Prahlad Modi stages dharna a...  10 hours ago   
4  You're alive because of PM Narendra Modi: Biha...  18 hours ago   

                                         Description  \
0  In redeveloping the Indian capital the prime m...   
1  PM Narendra Modi put 'tiranga' as the display ...   
2  My mother beats me for asking for a pencil: 6-...   
3  Prime Minister Narendra Modi's brother Prahlad...   
4  Even one of my relatives died in the coronavir...   

                                              Source  Negative  Positive  \
0  https://www.theatlantic.com/international/arch...     0.000    0.0000   
1  https://www.business-standard.com/article/curr...     0.000    0.0000   
2  https://eco

In [14]:
print(df.describe())

         Negative    Positive     Neutral  Aggregate Compound
count  144.000000  144.000000  144.000000          144.000000
mean     0.026771    0.092281    0.880962            0.151456
std      0.056876    0.101893    0.108268            0.301928
min      0.000000    0.000000    0.573000           -0.494350
25%      0.000000    0.000000    0.817875            0.000000
50%      0.000000    0.066250    0.898500            0.081275
75%      0.000000    0.145625    1.000000            0.309200
max      0.269000    0.427000    1.000000            0.865800


Top 5 Positive News

In [15]:
df2=df.nlargest(5,['Aggregate Compound'])
df2[['Title' ,'Description','Time','Aggregate Compound']]

,Title,Description,Time,Aggregate Compound
65,President Droupadi Mumru PM Narendra Modi Cong...,President Droupadi Mumru PM Narendra Modi Cong...,2 days ago,0.86580
32,Commonwealth Games 2022: PM Narendra Modi 'ela...,Prime Minister Narendra Modi has congratulated...,1 day ago,0.78265
98,Athletes will give their best: PM Modi wishes ...,PM Modi extended his best wishes to the Indian...,4 days ago,0.77225
21,PM Modi congratulates Mirabai Chanu says her s...,Prime Minister Narendra Modi on Saturday congr...,3 days ago,0.76145
22,PM Narendra Modi reacts to India`s Lawn Bowls ...,PM Narendra Modi reacts to India's Lawn Bowls ...,4 hours ago,0.75790


Top 5 negative News

In [16]:
df2=df.nsmallest(5,['Aggregate Compound'])
df2[['Title' ,'Description','Time','Aggregate Compound']]

,Title,Description,Time,Aggregate Compound
24,PM Narendra Modi condoles the loss of lives du...,New Delhi : The Prime Minister Narendra Modi h...,7 hours ago,-0.49435
56,PM Modi expresses condolences to families of d...,New Delhi [India] August 2 (ANI): Prime Minist...,23 hours ago,-0.44870
74,In Narendra Modi era no corrupt will go scot-f...,On Prime Minister Narendra Modi's watch no one...,4 days ago,-0.39185
122,In Modi era no corrupt will go scot-free: Trip...,On Prime Minister Narendra Modi's watch no one...,4 days ago,-0.39185
2,My mother beats me for asking for a pencil: 6-...,My mother beats me for asking for a pencil: 6-...,1 day ago,-0.38180
